In [ ]:
# ] add DataFrames CSV

In [ ]:
using Downloads
using DataFrames
using CSV

In [ ]:
using JuMP
using GLPK

In [ ]:
pim = Downloads.download("https://raw.githubusercontent.com/jump-dev/JuMP.jl/master/docs/src/tutorials/Getting%20started/data/passport-index-matrix.csv")
passportData = CSV.File(pim) |> DataFrame
# The values in each cell are as follows:
# * 3 = visa-free travel
# * 2 = eTA is required
# * 1 = visa can be obtained on arrival
# * 0 = visa is required
# * -1 is for all instances where passport and destination are the same

In [ ]:
for i in 1:DataFrames.nrow(passportData)
    for j in 2:DataFrames.ncol(passportData)
        if passportData[i, j] == -1 || passportData[i, j] == 3
            passportData[i, j] = 1
        else
            passportData[i, j] = 0
        end
    end
end
passportData
# The values in the cells now represent:
# * 1 = no visa required for travel
# * 0 = visa required for travel

In [ ]:
passportData[!,["Passport","Chile", "Brazil", "Italy"]]

In [ ]:
countries = passportData[!,"Passport"]

In [ ]:
model = JuMP.Model(GLPK.Optimizer)

@variable(model, x[j in countries], Bin)

@objective(model, Min, sum(x))

@constraint(model, [dest in countries], passportData[:, dest]' * x >= 1)

#@constraint(model, x["Chile"] == 1.0)
#@constraint(model, x["United States"] == 0.0)
;

In [ ]:
optimize!(model)

if JuMP.has_values(model)
    @show zval = JuMP.objective_value(model)
    xval = JuMP.value.(x)
    
    for i in eachindex(xval)
        if xval[i] ≈ 1.0
            @show countries[i]
        end
    end
    
    optimalPassport = [c for c in countries if xval[c] ≈ 1.0]
end
;

In [ ]:
sa = ["Passport"]
push!(sa, "Argentina")
push!(sa, "Bolivia")
push!(sa, "Brazil")
push!(sa, "Chile")
push!(sa, "Colombia")
push!(sa, "Ecuador")
push!(sa, "Guyana")
push!(sa, "Paraguay")
push!(sa, "Peru")
push!(sa, "Suriname")
push!(sa, "Uruguay")
push!(sa, "Venezuela")


In [ ]:
df_rows = [v in sa for v in passportData[!, :Passport]]
df2 = passportData[ df_rows, sa]
# println(df2)

In [ ]:
cou = df2[!,"Passport"]
m = JuMP.Model(GLPK.Optimizer)
@variable(m, x[j in cou], Bin)
@objective(m, Min, sum(x))
@constraint(m, [dest in cou], df2[:, dest]' * x >= 1)
@constraint(m, x["Chile"] == 1.0)
optimize!(m)
;

In [ ]:
if JuMP.has_values(m)
    @show zval = JuMP.objective_value(m)
    xval = JuMP.value.(x)
    
    for i in eachindex(xval)
        if xval[i] ≈ 1.0
            @show cou[i]
        end
    end
    
    optimalPassport = [c for c in cou if xval[c] ≈ 1.0]
end
;